In [1]:
import cv2
import os
import numpy as np
from PIL import Image
from shutil import copyfile
import random

In [8]:
trainval_2007 = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/ImageSets/Segmentation/trainval.txt'):
    trainval_2007.append(line.strip())
test_2007 = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/ImageSets/Segmentation/test.txt'):
    test_2007.append(line.strip())
all_2007 = trainval_2007 + test_2007
print(len(all_2007))

trainval_2012 = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2012/ImageSets/Segmentation/trainval.txt'):
    trainval_2012.append(line.strip())
test_2012 = []
all_2012 = trainval_2012
print(len(all_2012))

632
2913


In [9]:
person_2007_trainval = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/ImageSets/Main/person_trainval.txt'):
    data = line.strip().split()
    if data[1] == '1':
        person_2007_trainval.append(data[0])
person_2007_test = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/ImageSets/Main/person_test.txt'):
    data = line.strip().split()
    if data[1] == '1':
        person_2007_test.append(data[0])
person_2007 = person_2007_trainval + person_2007_test
print(len(person_2007))

person_2012_trainval = []
for line in open('/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2012/ImageSets/Main/person_trainval.txt'):
    data = line.strip().split()
    if data[1] == '1':
        person_2012_trainval.append(data[0])
person_2012 = person_2012_trainval
print(len(person_2012))

4015
4087


In [15]:
root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/'
file = open(root + "all_2007.txt", "w")
seg_2007 = []
for seg in all_2007:
    if seg in person_2007:
        seg_2007.append(seg)
        file.write(str(seg)+'\n')
file.close()
len(seg_2007)

250

In [16]:
root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/'
file = open(root + "all_2012.txt", "w")
seg_2012 = []
for seg in all_2012:
    if seg in person_2012:
        seg_2012.append(seg)
        file.write(str(seg)+'\n')
file.close()
len(seg_2012)

587

In [18]:
src_img_root = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/JPEGImages/'
src_mask_root = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/SegmentationClass/'
image_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/image/'
mask_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/mask/'
for seg in seg_2007:
    copyfile(src_img_root + seg + '.jpg', image_root + '2007_' + seg + '.jpg')
    copyfile(src_mask_root + seg + '.png', mask_root + '2007_' + seg + '.png')

In [19]:
src_img_root = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2012/JPEGImages/'
src_mask_root = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2012/SegmentationClass/'
image_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/image/'
mask_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/mask/'
for seg in seg_2012:
    copyfile(src_img_root + seg + '.jpg', image_root + seg + '.jpg')
    copyfile(src_mask_root + seg + '.png', mask_root + seg + '.png')

# VOC Mask to Binary Mask

In [33]:
mask_voc_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/mask_voc/'
mask_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/mask/'
mask_voc_list = os.listdir(mask_voc_root)
for img_name in mask_voc_list:
    img = Image.open(mask_voc_root + img_name)
    img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)  
    mask_img = np.zeros(img.shape[:2], np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            value = [img[i][j][0], img[i][j][1], img[i][j][2]]
            if value == [15, 15, 15]:
                mask_img[i][j] = 255
    cv2.imwrite(mask_root + img_name, mask_img)

# Annotation file

In [34]:
ann_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/annotations/'
ann_2007 = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2007/Annotations/'
for seg in seg_2007:
    copyfile(ann_2007 + seg + '.xml', ann_root + '2007_' + seg + '.xml')

In [36]:
ann_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/annotations/'
ann_2012 = '/media/arg_ws3/5E703E3A703E18EB/data/VOCdevkit/VOC2012/Annotations/'
for seg in seg_2012:
    copyfile(ann_2012 + seg + '.xml', ann_root + seg + '.xml')

# Trainval file

In [5]:
root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/'
ann_root = '/media/arg_ws3/5E703E3A703E18EB/data/person_seg/annotations/'
file_list = os.listdir(ann_root)
all_file = open(root + "all.txt", "w")
train_file = open(root + "train.txt", "w")
test_file = open(root + "val.txt", "w")
ratio = 0.9
count = 0
pick = []
while count < len(file_list):
    idx = random.randint(0, len(file_list)-1)
    if idx in pick:
        continue
    pick.append(idx)
    all_file.write(file_list[idx]+'\n')
    if count < len(file_list)*ratio:
        train_file.write(file_list[idx][:-4]+'\n')
    else:
        test_file.write(file_list[idx][:-4]+'\n')   
    count = count + 1
all_file.close()
train_file.close()
test_file.close()